In [ ]:
encoder_path = "./model/encoder/model.onnx"
decoder_path = "./model/decoder/model.onnx"
model_id = "openai/whisper-large-v3-turbo"

ExecutionProvider="QNNExecutionProvider"

In [ ]:
# reference: https://learn.microsoft.com/en-us/windows/ai/new-windows-ml/tutorial?tabs=python#acquiring-the-model-and-preprocessing
import subprocess
import json
import sys
import os
import onnxruntime as ort

def register_execution_providers():
    worker_script = os.path.abspath('winml.py')
    print(worker_script)
    result = subprocess.check_output([sys.executable, worker_script], text=True)
    paths = json.loads(result)
    for item in paths.items():
        try:
            ort.register_execution_provider_library(item[0], item[1])
        except Exception as e:
            print(f"Failed to register execution provider {item[0]}: {e}")

register_execution_providers()

In [ ]:
import sys
from pathlib import Path

NOTEBOOK_DIR = Path(__file__).parent if "__file__" in globals() else Path.cwd()
PROJECT_ROOT = NOTEBOOK_DIR.parents[1]
sys.path.insert(0, str(PROJECT_ROOT))

from qnn_app import HfWhisperAppWithSave, infer_audio

app = HfWhisperAppWithSave(encoder_path, decoder_path, model_id, ExecutionProvider, ort.OrtHardwareDeviceType.NPU)
audio_path = "../../data/librispeech_asr/test"
audio_file = next((os.path.join(d, f) for d, _, fs in os.walk(audio_path) for f in fs if f.endswith(".npy")), None)

save_data = False

infer_audio(app, model_id, audio_file, save_data)